In [1]:
import os.path
import sys
import django
sys.path.append('/home/galm/software/django/tmv/BasicBrowser')
sys.path.append('/home/max/software/django-tmv/tmv_mcc-apsis/BasicBrowser')
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "BasicBrowser.settings")
django.setup()
import scoping
from scoping.models import *

import pandas as pd

In [2]:
q = Query.objects.get(pk=10452)
p = q.project

dids = set(Doc.objects.filter(query=q).values_list('pk', flat=True))
print(len(dids))

49716


In [3]:
import statistics

django.db.connection.close()

def mode_na(x):
    try:
        return statistics.mode([y for y in x if y is not np.NaN])
    except:
        return np.NaN
    
columns = [
    'id',
    'content',
    'title',
    'wosarticle__de',
    'wosarticle__wc'
]

# Majority ratings
dos = DocOwnership.objects.filter(
    query=q, 
    relevant__gt=0,
    doc__content__iregex='\w'
)
dodf = pd.DataFrame.from_dict(dos.order_by('finish').values(
    'id','doc_id','user__username','relevant'
))
dodf.loc[dodf['relevant']==2,'relevant'] = 0
dodf.loc[dodf['relevant']==3,'relevant'] = np.NaN

seen_df = dodf.groupby('doc_id').agg(lambda x: mode_na(x)).reset_index()[['doc_id','relevant']].rename(columns={"doc_id":"id"})
seen_df['seen'] = 1
seen_df.loc[pd.isna(seen_df['relevant']),'seen'] = np.NaN

ddf = pd.DataFrame.from_dict(
    Doc.objects.filter(pk__in=seen_df['id']).values(*columns)
)
seen_df = pd.merge(ddf,seen_df)

seen_df.to_csv('data/0_labelled_documents.csv',index=False)
print(seen_df.shape)
seen_df.head()

(7263, 7)


,id,content,title,wosarticle__de,wosarticle__wc,relevant,seen
0,893971,Project premise Australian cities are forecast...,Towards a new development model for housing re...,Development; Greyfield; Housing; Model; Precin...,None,0.0,1.0
1,1445590,Margarita Island is one of the regions of Vene...,VEGETATION AND LAND USE IN NUEVA ESPARTA STATE...,None,[Ecology],0.0,1.0
2,623539,Phosphogypsum is the main waste generated by t...,An anomalous metal-rich phosphogypsum: Charact...,Leaching tests; Hazardousness classification; ...,"[Engineering, Environmental; Environmental Sci...",0.0,1.0
3,5023454,Hourly observations of temperature difference ...,Variation de la stabilité atmosphérique urbain...,None,None,0.0,1.0
4,5023430,The purpose of this study was to examine possi...,Risk Factors for Lung Cancer Among Nonsmoking ...,agricultural worker; air pollution; article; a...,None,0.0,1.0


In [4]:
import matplotlib.pyplot as plt
seen_df.groupby('relevant').size().plot.bar()

In [5]:
unseen_docs = Doc.objects.filter(
    id__in=dids,
    content__iregex='\w',
).exclude(pk__in=seen_df['id'])

unseen_df = pd.DataFrame.from_dict(list(unseen_docs.values(*columns)))
unseen_df = unseen_df[columns]
unseen_df['relevant'] = 0
unseen_df['seen'] = 0
unseen_df['random_sample'] = 0

print(unseen_df.shape)
unseen_df.to_csv('data/0_unlabelled_documents.csv', index=False)
unseen_df.head()

(42178, 8)


,id,content,title,wosarticle__de,wosarticle__wc,relevant,seen,random_sample
0,179,1. Biodiversity conservation of grasslands in ...,Influence of land use on plant community compo...,communal land; livestock grazing; plantation f...,[Biodiversity Conservation; Ecology],0,0,0
1,385,Ants are being widely studied in different Bra...,Effect of climatic variables and vine crops on...,Abiotic factors; Agroecosystems; Bioindicators...,None,0,0,0
2,1378,"Robinia pseudoacacia, invaded many countries a...",Black locust (Robinia pseudoacacia) beloved an...,Robinia pseudoacacia; Central Europe; History;...,[Forestry],0,0,0
3,1958,It is well-known that plasma HDL-C shows a neg...,[Introduction].,artifact; biography; Ottoman empire; theoretic...,None,0,0,0
4,2144,While both academics and politicians have long...,Feeding unrest: Disentangling the causal relat...,Africa; food prices; food security; instrument...,[International Relations; Political Science],0,0,0
